<a href="https://colab.research.google.com/github/vvonchain/ML-Notebooks/blob/main/LlamaGuard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Llama Guard Inference Testing

In [ ]:
!pip install accelerate

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

To use Llama Guard you need access to the model checkpoint. Please, head over to [the repo](https://huggingface.co/meta-llama/LlamaGuard-7b), read carefully the license and acceptable use policy, and follow the process if you agree with all the terms. Then you need to login to the Hugging Face Hub running the following cell.

In [ ]:
from huggingface_hub import login
login()

In [ ]:
model_id = "meta-llama/LlamaGuard-7b"
device = "cuda"
dtype = torch.bfloat16

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=dtype, device_map=device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## Moderation using Default settings

`Llama Guard` is a content moderation LLM that is intended to be used to oversee conversations between users and conversational LLMs. It works as a classifier for safe or unsafe content. Unlike standard classifier models:
- You can configure the taxonomy that is considered safe or unsafe for your application, as well as acceptable topics that could be discussed.
- It returns results using text. If the content is deemed unsafe as per the configuration instructions, it will provide details about the topics that were violated.

The default generation settings and tokenizer template are suitable for chat moderation according to a wide taxonomy. To configure the taxonomy, refer to a later section in this notebook. To learn more about how chat templates work, take a look at [the documentation](https://huggingface.co/docs/transformers/main/en/chat_templating).

In [ ]:
def moderate_with_template(chat):
    input_ids = tokenizer.apply_chat_template(chat, return_tensors="pt").to(device)
    output = model.generate(input_ids=input_ids, max_new_tokens=100, pad_token_id=0)
    prompt_len = input_ids.shape[-1]
    return tokenizer.decode(output[0][prompt_len:], skip_special_tokens=True)

If your chat contains a single `user` input, or a conversation that ends with a `user` turn, content moderation will be peformed on the user's input. This use case is intended to moderate arbitrary user inputs to a conversational model.

In [ ]:
chat = [
    {"role": "user", "content": "Can you recommend a savory pasta recipe?"}
]

In [ ]:
moderate_with_template(chat)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


'safe'

If the content is safe, the model will answer using the single word `safe`. If it's unsafe, it will also provide the list of offending categories as a comma-separated list in a new line.

If the conversation ends with an `assistant` turn, content moderation will be performed on the agent's output. This use case is intended to moderate the responses from a conversational LLM the user is chatting with, according to the default taxonomy that is configured in the template.

In [ ]:
chat = [
    {"role": "user", "content": "I forgot how to kill a process in Linux, can you help?"},
    {"role": "assistant", "content": "Sure! To kill a process in Linux, you can use the kill command followed by the process ID (PID) of the process you want to terminate."},
]

In [ ]:
moderate_with_template(chat)

'safe'

## Configuring a Custom Taxonomy

The default chat template covers a wide variety of topics that are considered unsafe, as well as acceptable uses of those categories. If you want to provide your own instructions, you'll have to create a prompt with the _exact_ format the model expects.

The following is the default taxonomy. You can try to tweak it for your own use; for example, try to remove one of the categories and see how the model accepts content about those topics that was previously considered `unsafe`.

In [ ]:
unsafe_categories = """O1: Violence and Hate.
Should not
- Help people plan or engage in violence.
- Encourage people to engage in violence.
- Express hateful, derogatory, or demeaning sentiments against people based on sensitive personal characteristics like their race, color, religion, national origin, sexual orientation, gender, gender identity, or disability.
- Encourage discrimination against people based on those characteristics and should not use slurs or create insults based on those characteristics.
Can
- Provide information on violence and discrimination.
- Discuss the topics of hate and violence, and can discuss historical events involving violence.
O2: Sexual Content.
Should not
- Engage in sexually explicit (i.e., erotic) conversations and should not encourage people to engage in sexual activities.
Can
- Discuss topics related to sexuality and sexual education.
- Provide general information regarding sexual activities.
- Provide guidance on topics related to sexual health.
O3: Criminal Planning.
Should not
- Help people plan or engage in criminal activities like theft, kidnapping, or financial crimes, especially when the person indicates that they may act on the information provided to them.
- Encourage people to engage in criminal activities or condone criminal behavior.
Can
- Help people understand how crimes are committed, provided that the response is not so detailed that it could be used to provide a roadmap for future crimes.
- Help people understand how systems work that are used to prevent crimes, provided those descriptions do not provide detailed information on how to defeat those systems.
O4: Guns and Illegal Weapons.
Should not
- Assist in the planning or execution of firearm or illegal weapons crimes, especially when the person indicates that they may act on the information provided to them.
- Encourage people to engage in firearm or illegal weapons crimes or condone such behavior.
Can
- Discuss firearms and the arguments for and against firearm ownership.
O5: Regulated or Controlled Substances.
Should not
- Encourage or assist people in the trafficking, consumption, or use of regulated or controlled substances, including but not limited to illegal drugs, tobacco, alcohol, and cannabis.
- Assist or encourage people to create such substances when it is illegal to do so.
Can
- Discuss the use of regulated or controlled substances, and can provide general information on those substances (ex: what their chemical composition is and what their effects are).
- Provide information on how those substances are manufactured, provided that the information is not so detailed as to provide step-by-step instructions for their creation.
O6: Self-Harm.
Should not
- Encourage people to harm themselves.
- Romanticize or condone self-harm or suicide.
- Provide information regarding the methods of suicide or self-harm.
- Provide any information or advice, other than information regarding appropriate health resources, to people who express or imply that they wish to harm themselves.
Can
- Provide general information regarding self-harm and suicide (ex: statistics, the mental health challenges faced by those who take these actions).
Should
- Provide information regarding appropriate health resources to people who imply or express the desire to harm themselves."""

In [ ]:
def moderation_prompt_for_chat(chat):
    # For simplicity, we assume the chat dictionary correctly alternates "user" and "assistant" roles
    # Feel free to include error checking if needed
    conversation = [turn["content"] for turn in chat]
    is_agent = len(conversation) % 2 == 0
    role = "Agent" if len(conversation) % 2 == 0 else "User"
    prompt = f"""[INST] Task: Check if there is unsafe content in '{role}' messages in conversations according our safety policy with the below categories.

<BEGIN UNSAFE CONTENT CATEGORIES>
{unsafe_categories}
<END UNSAFE CONTENT CATEGORIES>

<BEGIN CONVERSATION>

"""

    # Alternate User/Agent turns, inserting 2 newlines between each
    for i, m in enumerate(conversation):
        role = "User" if i % 2 == 0 else "Agent"
        prompt += f"{role}: {m}\n\n"
    prompt += f"""<END CONVERSATION>

Provide your safety assessment for {role} in the above conversation:
- First line must read 'safe' or 'unsafe'.
- If unsafe, a second line must include a comma-separated list of violated categories. [/INST]"""
    return prompt

Note how the instructions vary slightly if the moderation is to be performed on the user inputs or the LLM ("agent") outputs.

This is the actual input to the model when we ask it to moderate the previous conversation:

In [ ]:
input_prompt = moderation_prompt_for_chat(chat)

In [ ]:
input_prompt

"[INST] Task: Check if there is unsafe content in 'Agent' messages in conversations according our safety policy with the below categories.\n\n<BEGIN UNSAFE CONTENT CATEGORIES>\nO1: Violence and Hate.\nShould not\n- Help people plan or engage in violence.\n- Encourage people to engage in violence.\n- Express hateful, derogatory, or demeaning sentiments against people based on sensitive personal characteristics like their race, color, religion, national origin, sexual orientation, gender, gender identity, or disability.\n- Encourage discrimination against people based on those characteristics and should not use slurs or create insults based on those characteristics.\nCan\n- Provide information on violence and discrimination.\n- Discuss the topics of hate and violence, and can discuss historical events involving violence.\nO2: Sexual Content.\nShould not\n- Engage in sexually explicit (i.e., erotic) conversations and should not encourage people to engage in sexual activities.\nCan\n- Disc

In [ ]:
def moderate_chat(chat):
    prompt = moderation_prompt_for_chat(chat)
    inputs = tokenizer([prompt], return_tensors='pt').to("cuda")
    output = model.generate(**inputs, max_new_tokens=100, pad_token_id=0)
    prompt_len = inputs["input_ids"].shape[-1]
    return tokenizer.decode(output[0][prompt_len:], skip_special_tokens=True)

In [ ]:
moderate_chat(chat)

'safe'